# Projeto Final | Sistema de Controle Financeiro

%----------------------------------------------------------------------------%<br>
<br>
``Autores:`` Andrea Elias, Anthony Heimlich, Éverton Donato, Julia Midori e Luana Kruger  <br>
<br>
``Instituição:`` ADA Tech<br>
<br>
``Projeto:`` Santander Coders 2023.2<br>
<br>
``Descrição:`` Este código desenvolve um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.<br>
<br>
``Repositório GitHub:`` https://github.com/JuliaMidoriRW/Trabalho_Final_LogProgII_Grupo4 <br>
<br>

%----------------------------------------------------------------------------%

# Carregar bibliotecas e lista de registros

In [152]:
from datetime import datetime
# import json

registros = []

# Carregar funções

In [153]:
def criar_registro(tipo, valor, data):
    """Criar novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor

    Parameters
    ----------
    tipo: string
        Tipo da movimentação realizada. Os tipos podem ser:
            - Receita (valor numérico e armazenado normalmente), 
            - Despesas (valor positivo, mas armazenado como negativo),
            - Investimento (tem a informação de 'Montante' - calcular quanto o dinheiro rendeu desde o dia que foi investido)

    valor: float
        Valor do registro
    data: datetime
        Data do registro

    Returns
    -------

    """

    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
    montante = 0

    if tipo == 'Receita':
        valor = float(valor)
    elif tipo == 'Despesa':
        valor = -float(valor)
    elif tipo == 'Investimento':
        valor = float(valor)
        montante = valor * (1 + 0.02) ** ((datetime.now().date() - data_registro).days) - valor
    else:
        raise ValueError("Tipo de movimentação inválida.")
    
    registro = {'tipo': tipo, 'valor': valor, 'dia': dia, 'mes': mes, 'ano': ano}

    if montante > 0:
        registro['montante'] = montante
        
    registros.append(registro)

#     movimento = {
#         "tipo": tipo,
#         "valor": valor if tipo == "Receita" else -valor,
#         "data": data,
#     }

#     if tipo == "Investimento":
#         montante = calcula_rendimento(valor, data)
#         movimento["montante"] = montante

#     registros.append(movimento)

In [154]:
def ler_registros(chave,valor):
    """Consultar os registros por data, tipo ou valor

    Parameters
    ----------


    Returns
    -------

    """

    resultado = []

    for movimento in registros:
        if chave != "data":
            if movimento[chave] == valor:
                resultado.append(movimento)
        else:
            data_registro = datetime.strptime(valor, "%Y-%m-%d").date()
            dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
            if movimento["dia"] == dia and movimento["mes"] == mes and movimento["ano"] == ano:
                resultado.append(movimento)

    return resultado


In [155]:
def calcula_rendimento(valor, data):
    """Calcular montante.

    Parameters
    ----------
    valor: float
        valor do registro
    data: datetime
        data do registro

    Returns
    -------
    montante: float
        valor do rendimento

    """

    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    data_agora = datetime.now().date()

    dias = (data_agora - data_registro).days
    taxa = 0.01  # Exemplo: taxa de rendimento diária (1%)
    montante = valor * (1 + taxa) ** dias

    return montante

In [156]:
def atualizar_registro(indice, tipo, valor):
    """Atualiza o valor e o tipo do registro, a data deverá ser a de atualização do registro

    Parameters
    ----------


    Returns
    -------

    """
    
    if indice < len(registros):
        registro = registros[indice]
        if tipo:
            registro['tipo'] = tipo
        if valor:
            registro['valor'] = valor
            
        data_agora = datetime.now()
        dia, mes, ano = data_agora.day, data_agora.month, data_agora.year
        registro['dia'], registro['mes'], registro['ano'] = dia, mes, ano

        if registro['tipo'] == "Investimento":
            registro['montante'] = calcula_rendimento(registro['valor'], f"{ano}-{mes:02d}-{dia:02d}")

        

    
    # movimento = registros[indice]
    # movimento["tipo"] = tipo
    # movimento["valor"] = valor if tipo == "Receita" else -valor
    # movimento["data"] = data

In [157]:
def deletar_registro(indice):
    """Deletar o registro.

    Parameters
    ----------
    indice: int
        Indice do registro que será deletado:

    Returns
    -------

    """
    
    del registros[indice]

In [158]:
def atualiza_rendimento():
    """Atualiza os valores de rendimento sempre que chamada

    Parameters
    ----------

    Returns
    -------

    """
    
    for movimento in registros:
        if movimento["tipo"] == "Investimento":
            valor_inicial = -movimento["valor"]
            concatenar = str(movimento['ano']) + '-' + str(movimento['mes']) + '-' + str(movimento['dia'])
            data_investimento = datetime.strptime(concatenar, "%Y-%m-%d")
            movimento["montante"] = calcula_rendimento(valor_inicial, data_investimento.strftime("%Y-%m-%d"))

In [159]:
def exportar_relatorio(formato):
    """Exportar um relatorio final em csv ou json

    Parameters
    ----------
    formato: string
        Formato do relatório exportado ('.csv' ou '.json')

    Returns
    -------

    """
    
    if formato == "csv":
        with open("relatorio.csv", "w") as file:
            file.write("Tipo,Valor,Data,Montante\n")
            for movimento in registros:
                file.write(
                    f"{movimento['tipo']},{movimento['valor']},{movimento['data']},{movimento.get('montante', 0)}\n"
                )
    elif formato == "json":
        with open("relatorio.json", "w") as file:
            json.dump(registros, file, indent=2)

In [160]:
def agrupar_por(chave):
    """Função de agrupamento capaz de mostrar o total de valor baseado em alguma informação (mes, tipo...)

    Parameters
    ----------
    chave: string
        Informação base para o agrupamento ('tipo', 'mes', 'ano', 'data')

    Returns
    -------
    resultado: dict
        Dicionário com o registro agrupado por chave

    """
        
    resultado = {}
    for movimento in registros:
        valor = movimento["valor"]
        if chave == "tipo":
            chave_valor = movimento["tipo"]
        elif chave == "mes":
            chave_valor = movimento["data"].split("-")[1]
        elif chave == "ano":
            chave_valor = movimento["data"].split("-")[0]
        else:
            chave_valor = movimento["data"]

        resultado[chave_valor] = resultado.get(chave_valor, 0) + valor

    return resultado

# Testes executados

### Criar registro

In [161]:
criar_registro("Receita", 100, "2022-01-01")
criar_registro("Despesa", 50, "2022-01-02")
criar_registro("Investimento", 200, "2022-01-03")
criar_registro("Receita", 1000, "2022-01-01")
criar_registro("Despesa", 90, "2022-01-02")
criar_registro("Investimento", 200, "2023-05-03")

registros

[{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'Despesa', 'valor': -50.0, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 1,
  'ano': 2022,
  'montante': 490859768.0409117},
 {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'Despesa', 'valor': -90.0, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 5,
  'ano': 2023,
  'montante': 32903.530223710666}]

### Atualizar registro

In [162]:
atualizar_registro(0, "Receita", 150)

print("Registros por tipo:", ler_registros("tipo", "Receita"))
# print("Registros por data:", ler_registros("data", "2022-01-02"))

Registros por tipo: [{'tipo': 'Receita', 'valor': 150, 'dia': 16, 'mes': 1, 'ano': 2024}, {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}]


### Deletar registro

In [163]:
deletar_registro(1)

### Atualizar rendimento

In [164]:
atualiza_rendimento()
registros

[{'tipo': 'Receita', 'valor': 150, 'dia': 16, 'mes': 1, 'ano': 2024},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 1,
  'ano': 2022,
  'montante': -324945.6415169395},
 {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'Despesa', 'valor': -90.0, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 5,
  'ano': 2023,
  'montante': -2605.820111372499}]

### Exportar relatorio .CSV

In [165]:
exportar_relatorio("csv")
# exportar_relatorio("json")

KeyError: 'data'

### Agrupar por tipo

In [ ]:
print("Agrupado por tipo:", agrupar_por("tipo"))
print("Agrupado por mês:", agrupar_por("mes"))
print("Agrupado por ano:", agrupar_por("ano"))
print("Agrupado por data:", agrupar_por("data"))

Registros por tipo: [{'tipo': 'Receita', 'valor': 150, 'data': '2022-01-05'}]
Agrupado por tipo: {'Receita': 150}
Agrupado por mês: {'01': 150}
Agrupado por ano: {'2022': 150}
Agrupado por data: {'2022-01-05': 150}


### Consultas por tipo

In [ ]:
print("Pesquisa por tipo Receita:", ler_registros("tipo",'Receita'))
print("Pesquisa por tipo Despesa:", ler_registros("tipo",'Despesa'))
print("Pesquisa por tipo Investimento:", ler_registros("tipo",'Investimento'))

print("Pesquisa por valor:", ler_registros("valor",100))
print("Pesquisa por valor:", ler_registros("valor",-50))

print("Pesquisa por data:", ler_registros("data","2022-01-01"))
print("Pesquisa por data:", ler_registros("data","2023-05-03"))

Pesquisa por tipo Receita: [{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}]
Pesquisa por tipo Despesa: [{'tipo': 'Despesa', 'valor': -50.0, 'dia': 2, 'mes': 1, 'ano': 2022}, {'tipo': 'Despesa', 'valor': -90.0, 'dia': 2, 'mes': 1, 'ano': 2022}]
Pesquisa por tipo Investimento: [{'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 1, 'ano': 2022, 'montante': 490859768.0409117}, {'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 32903.530223710666}]
Pesquisa por valor: [{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022}]
Pesquisa por valor: [{'tipo': 'Despesa', 'valor': -50.0, 'dia': 2, 'mes': 1, 'ano': 2022}]
Pesquisa por data: [{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}]
Pesquisa por data: [{'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'me